In [1]:
# Importing libraries...
import time
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

from pid import PID

# Init client
client = RemoteAPIClient()  # Client object
sim = client.getObject("sim")  # Simulation object

In [2]:
class Individual:
    def __init__(
        self,
        genome = np.array([])
    ):
        self.genome = genome
        self.metrics = np.array([])
        self.fitness = -np.inf

In [3]:
def init_population(population_size, genome_lenght):
    return np.random.uniform(0.0, 10.0, (population_size, genome_lenght))


def crossover(genome_x, genome_y, crossover_rate):
    if np.random.rand() < crossover_rate:
        c = np.random.randint(0, len(genome_x) - 1)

        return np.concatenate((genome_x[:c], genome_y[c:])), np.concatenate((genome_y[:c], genome_x[c:]))
    else:
        return genome_x, genome_y


def mutate(genome, mutation_rate):
    new_genome = genome * np.random.uniform(1.0 - mutation_rate, 1.0 + mutation_rate, len(genome)) # It can drop or rise by 25%
    return new_genome


def get_fitness(metrics, weights):
    return np.dot(metrics, weights)


def pick_fittest(population, fitness_values):
    return population[np.argsort(fitness_values)][-2:] # Pick fittest parents

In [4]:
'''class Population:
    def __init__(
        self,
        individual_type = Individual,
        size=2,
        crossover_rate=0.0,
        mutation_rate=0.0,
        weights=np.array([])
    ):
        self.individual_type = individual_type
        self.size = size
        self.crossover_rate= crossover_rate,
        self.mutation_rate= mutation_rate,
        self.weights= weights

        self.individuals = []

    def init_population(self, lower_guess, higher_guess):
        # Create a list of random genome individuals with the size of the population
        self.individuals = [self.individual_type(
            genome=np.random.uniform(lower_guess, higher_guess, len(self.individual_type.genome))
        )

        for _ in range(self.size)]

    def crossover(self, genome_x, genome_y):
        # Crossover
        if np.random.rand() < self.crossover_rate: 
            c = np.random.randint(0, len(self.individual_type.genome) - 1)

            return np.concatenate((genome_x[:c], genome_y[c:])), np.concatenate((genome_y[:c], genome_x[c:]))

        # Don't crossover
        else:
            return genome_x, genome_y
        
    
    def mutate(self):
        for individual in self.individuals:
            # Drop or rise parameters by mutation_rate%
            individual.genome *= np.random.uniform(1.0 - self.mutation_rate, 1.0 + self.mutation_rate, len(self.individual_type.genome))


    def update_fitness(self):
        for individual in self.individuals:
            individual.fitness = np.dot(individual.metrics, self.weights)


    def order_by_fittest(self):
        fitness_values = [individual.fitness for individual in self.individuals]
        sorted_indexes = np.argsort(fitness_values)
        self.individuals = [self.individuals[index] for index in sorted_indexes]
        '''

"class Population:\n    def __init__(\n        self,\n        individual_type = Individual,\n        size=2,\n        crossover_rate=0.0,\n        mutation_rate=0.0,\n        weights=np.array([])\n    ):\n        self.individual_type = individual_type\n        self.size = size\n        self.crossover_rate= crossover_rate,\n        self.mutation_rate= mutation_rate,\n        self.weights= weights\n\n        self.individuals = []\n\n    def init_population(self, lower_guess, higher_guess):\n        # Create a list of random genome individuals with the size of the population\n        self.individuals = [self.individual_type(\n            genome=np.random.uniform(lower_guess, higher_guess, len(self.individual_type.genome))\n        )\n\n        for _ in range(self.size)]\n\n    def crossover(self, genome_x, genome_y):\n        # Crossover\n        if np.random.rand() < self.crossover_rate: \n            c = np.random.randint(0, len(self.individual_type.genome) - 1)\n\n            return np

In [5]:
class GoPiGo3(Individual):
    def __init__(
        self,
        # CoppeliaSim Handle
        handle_name = "/GoPiGo3",
        target_handle_name = "/Target",
        # Controllers
        genome = np.zeros(6)
    ):
        # Individual's inherited properties
        Individual.__init__(
            self, 
            genome=genome
        )

        # Handles
        self.handle = sim.getObject(handle_name)
        self.target_handle = sim.getObject(target_handle_name)
        self.motor_L_handle = sim.getObject(handle_name + "/LeftJoint")
        self.motor_R_handle = sim.getObject(handle_name + "/RightJoint")

        # Controllers
        self.OrientationController = PID(
            K_p=genome[0], 
            K_i=genome[1], 
            K_d=genome[2], 
            dt=sim.getSimulationTimeStep()
        )
        self.VelocityController = PID(
            K_p=genome[3], 
            K_i=genome[4], 
            K_d=genome[5], 
            dt=sim.getSimulationTimeStep()
        )

        # Differential Drive Model Parameters
        self.r = 0.2823
        self.s = 0.120

        # Metrics
        self.not_converged = True
        self.convergence_time = np.inf
        self.convergence_distance = np.inf
        self.target_distance = np.inf
        self.max_roll = 0.0
        self.max_pitch = 0.0

    def sense_and_actuate(self):
        timestamp = sim.getSimulationTime()

        # Getting GoPiGo3 Differential Drive Model Position
        position = np.mean(
            [
                sim.getObjectPosition(self.motor_L_handle), 
                sim.getObjectPosition(self.motor_R_handle)
            ],
            axis=0,
        )[:2]

        # Vectors and distances
        front_vector = np.array(sim.getObjectMatrix(self.handle)).reshape(3, 4)[:2, 0]
        roll, pitch, _ = np.abs(sim.getObjectOrientation(self.handle))
        path_vector = np.array(sim.getObjectPosition(self.target_handle))[:2] - position
        target_distance = np.linalg.norm(path_vector)
        target_vector = path_vector / target_distance

        # Error signals
        orientation_error = np.cross(front_vector, target_vector)
        position_error = np.dot(front_vector, target_vector) * target_distance

        # Controllers
        theta_dot = self.OrientationController.get_output(orientation_error)
        v_B = self.VelocityController.get_output(position_error)

        # Differential Drive Model
        theta_dot_L, theta_dot_R = np.linalg.inv(
            [
                [self.r / 2.0, self.r / 2.0], 
                [-self.r / self.s, self.r / self.s]
            ]
        ) @ np.array([v_B, theta_dot])

        # Update target distance
        self.target_distance = target_distance
        
        # Update max roll and max pitch
        if self.max_roll < roll: 
            self.max_roll = roll
        if self.max_pitch < pitch: 
            self.max_pitch = pitch

        # Convergence criteria
        self.not_converged = self.target_distance > 0.05

        # Stop condition
        if self.not_converged:
            # Move joints
            sim.setJointTargetVelocity(self.motor_L_handle, theta_dot_L)
            sim.setJointTargetVelocity(self.motor_R_handle, theta_dot_R)

            # Update convergence time
            self.convergence_time = timestamp

        else:
            # Move joints
            sim.setJointTargetVelocity(self.motor_L_handle, 0.0)
            sim.setJointTargetVelocity(self.motor_R_handle, 0.0)


    def get_metrics(self):
        # Returning vectorized performance parameters
        return np.array(
            [
                self.not_converged, 
                self.convergence_time, 
                self.target_distance,
                self.max_roll,
                self.max_pitch
            ]
        )

In [6]:
# Resetting Individuals
def reset_individuals(population_size):
    while True:
        try:
            sim.removeModel(sim.getObject(f"/Individual[1]"))
        except:
            break

    base_handle = sim.getObject("/Individual")
    for _ in range(population_size - 1):
        sim.copyPasteObjects([base_handle], 1)

In [7]:
# Simulation batch
simulation_time = 5.0
target_lower_bound = 0.1
target_higher_bound = 1.0

generations = 50
population_size = 10
crossover_rate = 0.10
mutation_rate = 0.25
weights = np.array([-40.0, -1.0, -20.0, -50.0, -50.0])

population = init_population(population_size, 6)
fitness_values = None

reset_individuals(population_size)

for generation, r in enumerate(np.linspace(target_lower_bound, target_higher_bound, generations)):
    print(f"Generation {generation}: Target at {r} m")

    # Set target to random position for this generation to pursue     
    theta = np.random.uniform(0.0, 2 * np.pi)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
        
    sim.setObjectPosition(
        sim.getObject('/Target'),
        [x, y, 0.05]
    )

    robot_population = [
        GoPiGo3(
            handle_name=f"/Individual[{i}]/GoPiGo3",
            target_handle_name="/Target",
            genome=genome
        )
        
        for i, genome in enumerate(population)
    ]

    try:
        sim.setStepping(True)  # Trigger simulation step manually

        # Start simulation
        sim.startSimulation()

        while sim.getSimulationTime() < simulation_time:
            for robot in robot_population:
                
                robot.sense_and_actuate()

            sim.step() # Call for next step

        # Stop simulation
        sim.stopSimulation()
        time.sleep(1.0) # Wait for simulation to stop

    except:
        pass
    
    # Get fitnes of every individual of the population
    fitness_values = [get_fitness(robot.get_metrics(), weights) for robot in robot_population]

    for robot, fitness_value in zip(robot_population, fitness_values):
        metrics = robot.get_metrics()
        
        print(f"Not converged: {robot.not_converged}") 
        print(f"Convergence time: {robot.convergence_time:.2f}") 
        print(f"Target distance: {robot.target_distance:.2f}") 
        print(f"Max roll: {robot.max_roll:.3f}") 
        print(f"Max pitch: {robot.max_pitch:.3f}") 
        print()
        print(robot.genome)
        print()
        print(f"Fitness: {fitness_value:.2f}", '\n\n')

    # Pick fittest and have them make them have offspring
    parent_x, parent_y = pick_fittest(population, fitness_values)

    # If it is on last generation, do not crossover and mutate
    if generation >= generations - 1:
        break

    # Make a new population based on variants of the most adapted individuals 
    new_population = []
    offspring_x, offspring_y = crossover(parent_x, parent_y, crossover_rate)

    for _ in range(population_size // 2):
        new_population.append(mutate(offspring_x, mutation_rate))
        new_population.append(mutate(offspring_y, mutation_rate))

    # Update population
    population = np.array(new_population)

Generation 0: Target at 0.1 m
Not converged:  False
Convergence time:  1.6000000000000008
Target distance:  0.049605522527326966
Max roll:  0.00024280780897672173
Max pitch:  0.0005752803649910943

[0.20373496 3.83485157 8.98391597 8.1051372  1.34480656 3.04559154]

Fitness:  -2.6330148592449305 


Not converged:  False
Convergence time:  1.1000000000000003
Target distance:  0.04880109614850903
Max roll:  0.00013465438639084255
Max pitch:  0.00019127600176248262

[5.29817461 8.22835153 1.08327509 5.96216807 2.90710055 5.43013593]

Fitness:  -2.0923184423778474 


Not converged:  False
Convergence time:  1.3500000000000005
Target distance:  0.0486972692906385
Max roll:  0.00013981705415301872
Max pitch:  0.0007842952826232058

[0.3253004  5.84660145 2.21443053 6.50404616 6.32511172 5.23835712]

Fitness:  -2.3701510026515815 


Not converged:  False
Convergence time:  1.5500000000000007
Target distance:  0.049631834639708425
Max roll:  0.00021221275458384745
Max pitch:  0.000590951604129

In [13]:
# Create the controller
genome = pick_fittest(population, fitness_values)[-1] # Pick fittest genome

reset_individuals(1)

robot = GoPiGo3(
    handle_name="/Individual/GoPiGo3",
    target_handle_name="/Target",
    genome=genome
)

try:
    sim.setStepping(True)  # Trigger simulation step manually

    # Start simulation
    sim.startSimulation()
    
    while not sim.getSimulationStopping():
        robot.sense_and_actuate()
        sim.step()

    # Stop simulation
    sim.stopSimulation()

except:
    pass